In [11]:
import pandas as pd
import numpy as np

# --- 1. CSV-Datei laden mit korrekter Dezimaltrennzeichen-Behandlung ---
file_path = 'data/kritiken_bereinigt.csv' # Dein Dateiname
try:
    df = pd.read_csv(file_path, decimal=',', low_memory=False)
    print(f"Datensatz '{file_path}' erfolgreich geladen. Anzahl der Zeilen: {len(df)}")
except FileNotFoundError:
    print(f"Fehler: Die Datei '{file_path}' wurde nicht gefunden. Bitte überprüfe den Pfad.")
    exit()

# --- 2. Sicherstellen, dass die 'Bewertung'-Spalte numerisch ist ---
if 'Bewertung' in df.columns:
    df['Bewertung'] = pd.to_numeric(df['Bewertung'], errors='coerce')
    print("\n'Bewertung'-Spalte wurde erfolgreich in numerisches Format konvertiert.")
    if df['Bewertung'].isnull().sum() > 0:
        print(f"Warnung: {df['Bewertung'].isnull().sum()} nicht-numerische Einträge in 'Bewertung' wurden zu NaN konvertiert.")
else:
    print(f"Fehler: Spalte 'Bewertung' wurde im DataFrame nicht gefunden. Bitte überprüfe den Spaltennamen.")

# --- 3. Potenzielle leere Strings in NaN umwandeln für alle Object-Spalten ---
# Wir überprüfen alle Spalten vom Typ 'object' (Strings), da diese leere Strings enthalten könnten.
object_columns = df.select_dtypes(include='object').columns

for col in object_columns:
    # Ersetzt leere Strings oder Strings, die nur aus Leerzeichen bestehen, durch np.nan
    df[col] = df[col].replace(r'^\s*$', np.nan, regex=True)

print("\nLeere Strings in Objektspalten wurden zu NaN konvertiert.")

# --- 4. Fehlende Werte für ALLE Spalten zählen und Prozentsätze berechnen ---
print("\n--- Analyse fehlender Werte für ALLE Spalten ---")

missing_counts_all = df.isnull().sum()
missing_percentages_all = (missing_counts_all / len(df)) * 100

print("\nAnzahl der fehlenden Werte pro Spalte:")
print(missing_counts_all)

print("\nProzentsatz der fehlenden Werte pro Spalte:")
print(missing_percentages_all)

print("\n--- Info über Datentypen nach allen Bereinigungen ---")
print(df.info())

print("\n--- Erste Zeilen des DataFrames nach allen Bereinigungen (zur Überprüfung) ---")
print(df.head())

Datensatz 'data/kritiken_bereinigt.csv' erfolgreich geladen. Anzahl der Zeilen: 107163

'Bewertung'-Spalte wurde erfolgreich in numerisches Format konvertiert.

Leere Strings in Objektspalten wurden zu NaN konvertiert.

--- Analyse fehlender Werte für ALLE Spalten ---

Anzahl der fehlenden Werte pro Spalte:
URL             0
Bewertung       0
Kritik          0
titel           0
laufzeit      820
genres         88
regie         152
drehbuch      818
besetzung    5191
dtype: int64

Prozentsatz der fehlenden Werte pro Spalte:
URL          0.000000
Bewertung    0.000000
Kritik       0.000000
titel        0.000000
laufzeit     0.765189
genres       0.082118
regie        0.141840
drehbuch     0.763323
besetzung    4.844023
dtype: float64

--- Info über Datentypen nach allen Bereinigungen ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107163 entries, 0 to 107162
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   URL   

In [12]:
import numpy as np

# --- 1 Fehlende Werte in anderen Spalten auffüllen ---
# Definieren der Spalten, die aufgefüllt werden sollen und ihrer spezifischen Platzhalter
columns_to_fill_with_placeholder = {
    'genres': 'Unbekanntes Genre',
    'regie': 'Unbekannter Regisseur',
    'drehbuch': 'Unbekannter Drehbuchautor',
    'besetzung': 'Unbekannte Besetzung'
}

print("\nFülle fehlende Werte in den angegebenen Spalten auf...")
for col, placeholder in columns_to_fill_with_placeholder.items():
    if col in df.columns:
        # Hier nutzen wir .fillna(), da die leeren Strings bereits in NaN konvertiert wurden
        df[col] = df[col].fillna(placeholder)
        print(f"- Spalte '{col}' mit '{placeholder}' aufgefüllt.")
    else:
        print(f"Warnung: Spalte '{col}' nicht im DataFrame gefunden, übersprungen.")

# Optional: Überprüfung der Missing Values nach dem Auffüllen (sollte 0 Missing Values anzeigen für diese Spalten)
print("\nÜberprüfung der Missing Values nach dem Auffüllen:")
print(df[list(columns_to_fill_with_placeholder.keys())].isnull().sum())


Fülle fehlende Werte in den angegebenen Spalten auf...
- Spalte 'genres' mit 'Unbekanntes Genre' aufgefüllt.
- Spalte 'regie' mit 'Unbekannter Regisseur' aufgefüllt.
- Spalte 'drehbuch' mit 'Unbekannter Drehbuchautor' aufgefüllt.
- Spalte 'besetzung' mit 'Unbekannte Besetzung' aufgefüllt.

Überprüfung der Missing Values nach dem Auffüllen:
genres       0
regie        0
drehbuch     0
besetzung    0
dtype: int64


In [13]:
import re

# --- 2.1 Funktion zur Konvertierung der Laufzeit ---
def convert_laufzeit_to_minutes(laufzeit_str):
    if pd.isna(laufzeit_str):
        return np.nan

    laufzeit_str = str(laufzeit_str).strip()
    total_minutes = 0

    hours_match = re.search(r'(\d+)\s*(?:Std|St|h)\.?', laufzeit_str, re.IGNORECASE)
    if hours_match:
        total_minutes += int(hours_match.group(1)) * 60

    minutes_match = re.search(r'(\d+)\s*(?:Min|M|min)\.?', laufzeit_str, re.IGNORECASE)
    if minutes_match:
        total_minutes += int(minutes_match.group(1))

    if total_minutes == 0 and re.fullmatch(r'\d+', laufzeit_str):
         total_minutes = int(laufzeit_str)

    return total_minutes if total_minutes > 0 or laufzeit_str == '0' else np.nan

print("\nVerarbeite die 'laufzeit'-Spalte...")
if 'laufzeit' in df.columns:
    df['laufzeit_minuten'] = df['laufzeit'].apply(convert_laufzeit_to_minutes)

    # --- 2.2 Fehlende Werte in 'laufzeit_minuten' mit dem Median auffüllen ---
    median_laufzeit = df['laufzeit_minuten'].median()
    df['laufzeit_minuten'] = df['laufzeit_minuten'].fillna(median_laufzeit)

    df['laufzeit_minuten'] = pd.to_numeric(df['laufzeit_minuten'], errors='coerce')

    print(f"- 'laufzeit' in 'laufzeit_minuten' (numerisch) umgewandelt.")
    print(f"- Fehlende Laufzeiten mit Median ({median_laufzeit:.0f} Minuten) aufgefüllt.")
    print(f"Anzahl fehlender Werte in 'laufzeit_minuten': {df['laufzeit_minuten'].isnull().sum()}")
    print("\nErste Zeilen von 'laufzeit' und 'laufzeit_minuten':")
    print(df[['laufzeit', 'laufzeit_minuten']].head())
else:
    print("Fehler: Spalte 'laufzeit' wurde nicht im DataFrame gefunden.")


Verarbeite die 'laufzeit'-Spalte...
- 'laufzeit' in 'laufzeit_minuten' (numerisch) umgewandelt.
- Fehlende Laufzeiten mit Median (110 Minuten) aufgefüllt.
Anzahl fehlender Werte in 'laufzeit_minuten': 0

Erste Zeilen von 'laufzeit' und 'laufzeit_minuten':
         laufzeit  laufzeit_minuten
0  1 Std. 56 Min.             116.0
1  1 Std. 56 Min.             116.0
2  1 Std. 56 Min.             116.0
3  1 Std. 23 Min.              83.0
4  1 Std. 23 Min.              83.0


In [14]:
import nltk

print("Starte NLTK-Download für 'wordnet'...")
nltk.download('wordnet')
print("Starte NLTK-Download für 'stopwords'...")
nltk.download('stopwords')
print("Starte NLTK-Download für 'punkt'...")
nltk.download('punkt')

print("\nAlle benötigten NLTK-Ressourcen sollten jetzt installiert sein.")

Starte NLTK-Download für 'wordnet'...
Starte NLTK-Download für 'stopwords'...
Starte NLTK-Download für 'punkt'...

Alle benötigten NLTK-Ressourcen sollten jetzt installiert sein.


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rusla\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rusla\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rusla\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [17]:
import pandas as pd
import numpy as np
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Initialisiere Lemmatizer und Stopwords
# Diese Initialisierungen sind nur dann erfolgreich, wenn die Daten gefunden werden.
try:
    lemmatizer = WordNetLemmatizer()
    german_stopwords = set(stopwords.words('german'))
    print("NLTK Lemmatizer und Stopwords erfolgreich geladen.")
except LookupError:
    print("FEHLER: NLTK-Ressourcen (WordNet, Stopwords) konnten nicht geladen werden. Bitte sicherstellen, dass 'nltk.download('wordnet')' und 'nltk.download('stopwords')' erfolgreich waren.")
    # Du könntest hier auch ein `exit()` oder `raise` hinzufügen, um weitere Fehler zu vermeiden.

# --- Funktion zur Textbereinigung und Lemmatisierung ---
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'https?://\S+|www\.\S+', '', text) # URLs entfernen
    text = re.sub(r'<.*?>', '', text) # HTML-Tags entfernen
    text = re.sub(f'[{re.escape(string.punctuation)}]', '', text) # Satzzeichen entfernen
    text = re.sub(r'\d+', '', text) # Zahlen entfernen
    text = re.sub(r'\s+', ' ', text).strip() # Mehrere Leerzeichen zu einem reduzieren & trimmen

    tokens = text.split()
    # Sicherstellen, dass der Lemmatizer und die Stopwords geladen sind
    if 'lemmatizer' in globals() and 'german_stopwords' in globals():
        cleaned_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in german_stopwords]
    else:
        # Fallback, falls NLTK-Initialisierung fehlgeschlagen ist (sollte dank 'raise' nicht passieren)
        cleaned_tokens = tokens
    return ' '.join(cleaned_tokens)

NLTK Lemmatizer und Stopwords erfolgreich geladen.


In [18]:
# --- ID-Extraktion aus URL für das Haupt-DataFrame (Nutzerkritiken) ---
print("\nExtrahiere Film-ID aus der 'URL'-Spalte des Haupt-DataFrames...")
if 'URL' in df.columns:
    df['film_id'] = df['URL'].apply(lambda x: re.search(r'/kritiken/(\d+)\.html$', str(x)).group(1) if re.search(r'/kritiken/(\d+)\.html$', str(x)) else np.nan)
    df['film_id'] = pd.to_numeric(df['film_id'], errors='coerce')
    print("- 'film_id' im Haupt-DataFrame erstellt. Anzahl nicht extrahierter IDs (NaN):", df['film_id'].isnull().sum())
else:
    print("Fehler: 'URL'-Spalte nicht im Haupt-DataFrame gefunden. ID-Extraktion nicht möglich.")

# --- Textbereinigung für 'Kritik' (Nutzerkritiken) ---
print("\nBereinige und normalisiere Text in 'Kritik'...")
if 'Kritik' in df.columns:
    df['Kritik_cleaned'] = df['Kritik'].apply(clean_text)
    print("- 'Kritik'-Spalte bereinigt und in 'Kritik_cleaned' gespeichert.")
else:
    print("Fehler: 'Kritik'-Spalte nicht im DataFrame gefunden.")
    # Explicitly create an empty 'Kritik_cleaned' column if 'Kritik' doesn't exist
    df['Kritik_cleaned'] = ''
    print("  Leere 'Kritik_cleaned' Spalte hinzugefügt, um spätere Fehler zu vermeiden.")

# Now, before aggregation, ensure 'Kritik_cleaned' NaN values are handled
# This is crucial for the lambda function 'lambda x: ' '.join(x.dropna().astype(str))'
# which expects a string-like series. If it's all NaN, dropna() would result in an empty series.
if 'Kritik_cleaned' in df.columns: # This check is more of a safeguard here
    df['Kritik_cleaned'] = df['Kritik_cleaned'].fillna('') # Ensure NaNs are empty strings before aggregation

## Step 1: Aggregate User Reviews

print("\nStarte Aggregation der Nutzerkritiken und Filmdaten auf Film-Ebene...")

# Define aggregation for user reviews and other attributes
aggregation_dict_user_data = {
    'titel': ('titel', 'first'), # <-- Hier wird der Titel aggregiert
    'Kritik_cleaned': ('Kritik_cleaned', lambda x: ' '.join(x.dropna().astype(str))),
    'genres': ('genres', lambda x: ', '.join(x.dropna().astype(str).unique())),
    'regie': ('regie', lambda x: ', '.join(x.dropna().astype(str).unique())),
    'drehbuch': ('drehbuch', lambda x: ', '.join(x.dropna().astype(str).unique())),
    'besetzung': ('besetzung', lambda x: ', '.join(x.dropna().astype(str).unique())),
    'laufzeit_minuten': ('laufzeit_minuten', 'mean'),
    'Bewertung': ('Bewertung', 'mean'),
    'film_id': ('film_id', 'first'), # Keep one film_id for merging later
}

# Group by 'URL' and aggregate user data
df_movies = df.groupby('URL').agg(**aggregation_dict_user_data).reset_index()

# Rename the aggregated user review column
df_movies.rename(columns={'Kritik_cleaned': 'Kritik_user_aggregated'}, inplace=True)

# Clean up aggregated text columns (remove leading/trailing spaces and replace empty strings with NaN)
df_movies['Kritik_user_aggregated'] = df_movies['Kritik_user_aggregated'].str.strip()
df_movies['Kritik_user_aggregated'] = df_movies['Kritik_user_aggregated'].replace('', np.nan)

print(f"Aggregation der Nutzerdaten abgeschlossen. 'df_movies' erstellt mit {len(df_movies)} einzigartigen Filmen.")
print("Beispiel der ersten Zeilen des aggregierten Nutzerdaten-DataFrames:")
print(df_movies.head())


Extrahiere Film-ID aus der 'URL'-Spalte des Haupt-DataFrames...
- 'film_id' im Haupt-DataFrame erstellt. Anzahl nicht extrahierter IDs (NaN): 0

Bereinige und normalisiere Text in 'Kritik'...
- 'Kritik'-Spalte bereinigt und in 'Kritik_cleaned' gespeichert.

Starte Aggregation der Nutzerkritiken und Filmdaten auf Film-Ebene...
Aggregation der Nutzerdaten abgeschlossen. 'df_movies' erstellt mit 13062 einzigartigen Filmen.
Beispiel der ersten Zeilen des aggregierten Nutzerdaten-DataFrames:
                                             URL  \
0   http://www.filmstarts.de/kritiken/10053.html   
1   http://www.filmstarts.de/kritiken/10055.html   
2  http://www.filmstarts.de/kritiken/100673.html   
3   http://www.filmstarts.de/kritiken/10070.html   
4  http://www.filmstarts.de/kritiken/100714.html   

                                               titel  \
0                                            M*A*S*H   
1                           Die nackte Kanone 33 1/3   
2  The First Time - Dein e

In [20]:
## Step 2: Process Professional Reviews and Merge

kritik_pro_file_path = 'data/bereinigt_professionelle_kritiken.csv'
kritik_pro_id_column = 'film_id'
kritik_pro_text_column = 'review'

try:
    df_kritik_pro = pd.read_csv(kritik_pro_file_path)
    print(f"\nProfessionelle Kritiken '{kritik_pro_file_path}' erfolgreich geladen. Anzahl der Zeilen: {len(df_kritik_pro)}")

    if kritik_pro_id_column in df_kritik_pro.columns:
        df_kritik_pro[kritik_pro_id_column] = pd.to_numeric(df_kritik_pro[kritik_pro_id_column], errors='coerce')
        df_kritik_pro.dropna(subset=[kritik_pro_id_column], inplace=True)
        df_kritik_pro[kritik_pro_id_column] = df_kritik_pro[kritik_pro_id_column].astype(int)
        print(f"- ID-Spalte '{kritik_pro_id_column}' in 'kritik_pro' bereinigt.")
    else:
        print(f"Fehler: ID-Spalte '{kritik_pro_id_column}' nicht in professionellen Kritiken gefunden.")

    if kritik_pro_text_column in df_kritik_pro.columns:
        df_kritik_pro['kritik_pro_cleaned'] = df_kritik_pro[kritik_pro_text_column].apply(clean_text)
        print(f"- '{kritik_pro_text_column}' in 'kritik_pro_cleaned' bereinigt.")
        df_kritik_pro['kritik_pro_cleaned'] = df_kritik_pro['kritik_pro_cleaned'].replace(r'^\s*$', np.nan, regex=True)
        df_kritik_pro['kritik_pro_cleaned'] = df_kritik_pro['kritik_pro_cleaned'].fillna('Keine professionelle Kritik vorhanden')
    else:
        print(f"Fehler: Textspalte '{kritik_pro_text_column}' nicht in professionellen Kritiken gefunden.")

    # Drop original text column and keep only the cleaned one for merging
    df_kritik_pro = df_kritik_pro[[kritik_pro_id_column, 'kritik_pro_cleaned']]

    # Merge df_movies (with aggregated user data) with df_kritik_pro
    initial_df_movies_rows = len(df_movies)
    initial_df_movies_cols = len(df_movies.columns)

    df_movies = pd.merge(df_movies, df_kritik_pro,
                         left_on='film_id', right_on=kritik_pro_id_column, how='left')

    print(f"\n'df_movies' mit professionellen Kritiken zusammengeführt über 'film_id'.")
    print(f"Anzahl der Zeilen nach Merge (sollte gleich sein): {len(df_movies)} (vorher: {initial_df_movies_rows})")
    print(f"Anzahl der Spalten nach Merge: {len(df_movies.columns)} (vorher: {initial_df_movies_cols})")

    if 'kritik_pro_cleaned' in df_movies.columns:
        df_movies['kritik_pro_cleaned'] = df_movies['kritik_pro_cleaned'].fillna('Keine professionelle Kritik vorhanden')
        print("- Fehlende 'kritik_pro_cleaned' nach Merge aufgefüllt.")
        print(f"Anzahl fehlender Werte in 'kritik_pro_cleaned': {df_movies['kritik_pro_cleaned'].isnull().sum()}")
    else:
        print("Warnung: 'kritik_pro_cleaned' Spalte wurde nicht erfolgreich gemerged.")

    # Clean up temporary ID column from the professional reviews DataFrame
    if kritik_pro_id_column in df_movies.columns:
        df_movies.drop(columns=[kritik_pro_id_column], inplace=True)
    print("\nTemporäre ID-Spalte aus professionellen Kritiken entfernt.")


except FileNotFoundError:
    print(f"Fehler: Die Datei '{kritik_pro_file_path}' wurde nicht gefunden. Bitte überprüfe den Pfad.")
except Exception as e:
    print(f"Ein Fehler ist beim Laden, Bereinigen oder Mergen der professionellen Kritiken aufgetreten: {e}")

# Final Check of the aggregated DataFrame
print("\n--- Finaler Check des aggregierten DataFrames nach allen Bereinigungsschritten ---")
print(f"Aktuelle Anzahl der Zeilen im DataFrame 'df_movies': {len(df_movies)}")
print(df_movies.info())
print(df_movies.head())

print("\nBeispiel für bereinigte Kritiken (erste Zeilen):")
if not df_movies.empty and 'Kritik_user_aggregated' in df_movies.columns:
    print("Normal (Nutzerkritik, aggregiert): ", df_movies['Kritik_user_aggregated'].iloc[0])
if not df_movies.empty and 'kritik_pro_cleaned' in df_movies.columns:
    print("Professionell: ", df_movies['kritik_pro_cleaned'].iloc[0])


Professionelle Kritiken 'data/bereinigt_professionelle_kritiken.csv' erfolgreich geladen. Anzahl der Zeilen: 7953
- ID-Spalte 'film_id' in 'kritik_pro' bereinigt.
- 'review' in 'kritik_pro_cleaned' bereinigt.

'df_movies' mit professionellen Kritiken zusammengeführt über 'film_id'.
Anzahl der Zeilen nach Merge (sollte gleich sein): 13062 (vorher: 13062)
Anzahl der Spalten nach Merge: 11 (vorher: 10)
- Fehlende 'kritik_pro_cleaned' nach Merge aufgefüllt.
Anzahl fehlender Werte in 'kritik_pro_cleaned': 0

Temporäre ID-Spalte aus professionellen Kritiken entfernt.

--- Finaler Check des aggregierten DataFrames nach allen Bereinigungsschritten ---
Aktuelle Anzahl der Zeilen im DataFrame 'df_movies': 13062
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13062 entries, 0 to 13061
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   URL                     13062 non-null  object 
 1   titel          

In [22]:
# --- Daten speichern nach der Bereinigung ---
output_file_path = 'data/kritiken_bereinigt_und_gemerged.csv'
try:
    df_movies.to_csv(output_file_path, index=False, decimal='.') # 'index=False' verhindert das Speichern des Pandas-Index als Spalte
    print(f"\nBereinigter DataFrame erfolgreich gespeichert unter '{output_file_path}'")
except Exception as e:
    print(f"Fehler beim Speichern des DataFrames: {e}")


Bereinigter DataFrame erfolgreich gespeichert unter 'data/kritiken_bereinigt_und_gemerged.csv'


In [23]:
# Importe für Feature Engineering
from sklearn.preprocessing import MultiLabelBinarizer, MinMaxScaler # MinMaxScaler hinzugefügt
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack, csr_matrix
import pandas as pd
import numpy as np
from collections import Counter


print("Starte Feature Engineering (Speicheroptimierte Version - Nutzt df_movies und skaliert Laufzeit)...")

# --- 1. Multi-Hot-Encoding für kategorische Spalten ---
categorical_columns_multi_hot = ['genres', 'regie', 'drehbuch', 'besetzung']
encoded_features = []

# Schwellenwerte für die Anzahl der Top-Features pro Kategorie
MAX_GENRES = 50
MAX_REGIE = 1000
MAX_DREHBUCH = 1000
MAX_BESETZUNG = 1500

feature_limits = {
    'genres': MAX_GENRES,
    'regie': MAX_REGIE,
    'drehbuch': MAX_DREHBUCH,
    'besetzung': MAX_BESETZUNG
}


for col in categorical_columns_multi_hot:
    if col in df_movies.columns:
        list_of_labels_raw = df_movies[col].astype(str).apply(lambda x: [item.strip() for item in x.split(',') if item.strip()])

        all_labels = [label for sublist in list_of_labels_raw for label in sublist]
        label_counts = Counter(all_labels)

        top_labels = [label for label, count in label_counts.most_common(feature_limits.get(col, len(label_counts)))]

        list_of_labels_filtered = list_of_labels_raw.apply(
            lambda x: [label for label in x if label in top_labels]
        )
        list_of_labels_filtered = list_of_labels_filtered.apply(
            lambda x: x if x else [f'Unbekannte_{col}']
        )

        mlb = MultiLabelBinarizer()
        encoded_matrix = mlb.fit_transform(list_of_labels_filtered)
        encoded_features.append(csr_matrix(encoded_matrix))
        print(f"- '{col}' wurde Multi-Hot-Encoded. Anzahl der neuen Spalten (gefiltert): {mlb.classes_.shape[0]}")
    else:
        print(f"Warnung: Spalte '{col}' nicht für Multi-Hot-Encoding gefunden, übersprungen.")


# --- 2. TF-IDF-Vektorisierung für Textspalten ---
text_columns_for_tfidf = ['Kritik_user_aggregated', 'kritik_pro_cleaned']
tfidf_matrices = []

TFIDF_MAX_FEATURES = 10000 # Beibehaltung deines Wertes


for col in text_columns_for_tfidf:
    if col in df_movies.columns:
        df_movies[col] = df_movies[col].astype(str).fillna('leerer_text_placeholder')
        
        tfidf_vectorizer = TfidfVectorizer(max_features=TFIDF_MAX_FEATURES, min_df=5)
        tfidf_matrix = tfidf_vectorizer.fit_transform(df_movies[col])
        tfidf_matrices.append(tfidf_matrix)
        print(f"- '{col}' wurde TF-IDF-Vektorisiert. Anzahl der Features: {tfidf_matrix.shape[1]}")
    else:
        print(f"Warnung: Textspalte '{col}' nicht für TF-IDF gefunden, übersprungen.")


# --- 3. Numerische Features extrahieren und SKALIEREN ---
numerical_features = []
if 'laufzeit_minuten' in df_movies.columns:
    # Hier stellen wir nur sicher, dass keine NaNs mehr vorhanden sind.
    # df_movies['laufzeit_minuten'] = df_movies['laufzeit_minuten'].fillna(df_movies['laufzeit_minuten'].median())
    
    # Skalieren der Laufzeit-Minuten auf einen Bereich zwischen 0 und 1
    scaler = MinMaxScaler()
    # Wichtig: .values.reshape(-1, 1) ist notwendig für den MinMaxScaler
    scaled_laufzeit = scaler.fit_transform(df_movies['laufzeit_minuten'].values.reshape(-1, 1))
    numerical_features.append(scaled_laufzeit)
    print("- 'laufzeit_minuten' als numerisches Feature hinzugefügt und skaliert (MinMaxScaler).")
else:
    print("Warnung: Spalte 'laufzeit_minuten' nicht gefunden, numerische Feature-Extraktion übersprungen.")


# Konvertiere numerische Arrays zu Sparse Matrizen für hstack
numerical_sparse_matrices = [csr_matrix(arr) for arr in numerical_features]


# --- 4. Alle Features kombinieren ---
print("\nKombiniere alle Features zu einer einzigen Matrix...")
all_sparse_features = tfidf_matrices + numerical_sparse_matrices + encoded_features

# Erstelle die finale Feature-Matrix X
X = hstack(all_sparse_features).tocsr()

print(f"Finale Feature-Matrix 'X' erstellt mit Shape: {X.shape}")
print(f"Anzahl der Features (Spalten) in X: {X.shape[1]}")
print(f"Anzahl der Samples (Zeilen) in X: {X.shape[0]}")


# Erstelle die Zielvariable 'y' (bleibt hier nur für den Fall, dass sie benötigt wird)
if 'Bewertung' in df_movies.columns:
    y = df_movies['Bewertung'].values
    print(f"Zielvariable 'y' (Bewertung) erstellt mit Shape: {y.shape}")
else:
    print("Fehler: 'Bewertung'-Spalte nicht gefunden, Zielvariable 'y' konnte nicht erstellt werden.")
    y = None

print("\nFeature Engineering abgeschlossen.")
print("Du hast nun die Feature-Matrix 'X' und die Zielvariable 'y'.")

Starte Feature Engineering (Speicheroptimierte Version - Nutzt df_movies und skaliert Laufzeit)...
- 'genres' wurde Multi-Hot-Encoded. Anzahl der neuen Spalten (gefiltert): 34
- 'regie' wurde Multi-Hot-Encoded. Anzahl der neuen Spalten (gefiltert): 1001
- 'drehbuch' wurde Multi-Hot-Encoded. Anzahl der neuen Spalten (gefiltert): 1001
- 'besetzung' wurde Multi-Hot-Encoded. Anzahl der neuen Spalten (gefiltert): 1501
- 'Kritik_user_aggregated' wurde TF-IDF-Vektorisiert. Anzahl der Features: 10000
- 'kritik_pro_cleaned' wurde TF-IDF-Vektorisiert. Anzahl der Features: 10000
- 'laufzeit_minuten' als numerisches Feature hinzugefügt und skaliert (MinMaxScaler).

Kombiniere alle Features zu einer einzigen Matrix...
Finale Feature-Matrix 'X' erstellt mit Shape: (13062, 23538)
Anzahl der Features (Spalten) in X: 23538
Anzahl der Samples (Zeilen) in X: 13062
Zielvariable 'y' (Bewertung) erstellt mit Shape: (13062,)

Feature Engineering abgeschlossen.
Du hast nun die Feature-Matrix 'X' und die Zielv

In [24]:
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import numpy as np
from fuzzywuzzy import process 
from fuzzywuzzy import fuzz 

print("\nStarte NearestNeighbors Modelltraining und erstelle Empfehlungsfunktion (Nutzt df_movies)...")

# --- 1. NearestNeighbors Modell trainieren ---
k_neighbors = 20 # Anzahl der ähnlichsten Filme, die wir finden wollen.

try:
    nn_model = NearestNeighbors(n_neighbors=k_neighbors + 1, algorithm='brute', metric='cosine')
    nn_model.fit(X) # X ist Sparse-Feature-Matrix (jetzt basierend auf df_movies)
    print(f"NearestNeighbors Modell erfolgreich trainiert mit {k_neighbors} Nachbarn.")
except Exception as e:
    print(f"FEHLER beim Trainieren des NearestNeighbors Modells: {e}")
    print("Stelle sicher, dass die Feature-Matrix 'X' existiert und korrekt erstellt wurde.")
    raise



Starte NearestNeighbors Modelltraining und erstelle Empfehlungsfunktion (Nutzt df_movies)...
NearestNeighbors Modell erfolgreich trainiert mit 20 Nachbarn.


In [25]:
# --- 2. Empfehlungsfunktion erstellen (nutzt NearestNeighbors) ---
def get_recommendations_nn(movie_title, df_movies_original, nn_model, feature_matrix_X, num_recommendations=10):
    """
    Gibt die Top N der ähnlichsten Filme basierend auf einem gegebenen Filmtitel zurück,
    unter Verwendung eines vortrainierten NearestNeighbors Modells.
    Filtert den Originalfilm intelligent heraus.

    Args:
        movie_title (str): Der Titel des Films, für den Empfehlungen gesucht werden.
        df_movies_original (pd.DataFrame): Dein aggregierter DataFrame df_movies.
        nn_model (sklearn.neighbors.NearestNeighbors): Das vortrainierte NearestNeighbors Modell.
        feature_matrix_X (scipy.sparse.csr_matrix): Die Feature-Matrix (X).
        num_recommendations (int): Die Anzahl der zurückzugebenden Empfehlungen.

    Returns:
        pd.DataFrame: Ein DataFrame mit den empfohlenen Filmen und deren Ähnlichkeitsscore.
    """
    # Finde den Index des Originalfilms im df_movies_original DataFrame.
    original_movie_normalized_title = movie_title.lower().strip()
    # HIER: Suche direkt in df_movies_original['titel']
    movie_indices = df_movies_original[df_movies_original['titel'].str.lower().str.strip() == original_movie_normalized_title].index.tolist()

    if not movie_indices:
        print(f"Film '{movie_title}' nicht im Datensatz gefunden. Bitte überprüfen Sie den Titel.")
        return pd.DataFrame()

    # Wir nehmen den ersten gefundenen Index für die Abfrage
    query_movie_index = movie_indices[0]

    # Finde die k-nächsten Nachbarn.
    # Hier fragen wir num_recommendations + 1 Nachbarn ab, da der erste immer der Film selbst ist.
    distances, indices = nn_model.kneighbors(feature_matrix_X[query_movie_index], n_neighbors=num_recommendations + 1)

    recommended_movies_data = []
    # Beginne bei 1, um den Film selbst zu überspringen (Distanz 0)
    for i in range(1, len(indices.flatten())):
        idx = indices.flatten()[i] # Der tatsächliche Index im df_movies_original DataFrame
        
        # Konvertiere Distanz in Ähnlichkeit
        similarity_score = 1 - distances.flatten()[i]

        # Sammle die Informationen des empfohlenen Films.
        # HIER: iloc auf df_movies_original anwenden
        recommended_movies_data.append({
            'Titel': df_movies_original.iloc[idx]['titel'],
            'Ähnlichkeit': similarity_score,
            'Genres': df_movies_original.iloc[idx]['genres'],
            'Regie': df_movies_original.iloc[idx]['regie'],
            'Laufzeit (Min.)': df_movies_original.iloc[idx]['laufzeit_minuten'],
            'Bewertung (Original)': df_movies_original.iloc[idx]['Bewertung']
        })
    
    return pd.DataFrame(recommended_movies_data)

print("Empfehlungsfunktion 'get_recommendations_nn' erfolgreich erstellt.")

Empfehlungsfunktion 'get_recommendations_nn' erfolgreich erstellt.


In [26]:
# --- Beispielaufruf der Empfehlungsfunktion ---
example_movie_title = 'Avatar 2: The Way Of Water' # Ein Filmtitel aus deinem Datensatz
print(f"\nGeneriere Empfehlungen für: '{example_movie_title}'")
# HIER: df durch df_movies ersetzt
recommended_movies = get_recommendations_nn(example_movie_title, df_movies, nn_model, X, num_recommendations=5)

if not recommended_movies.empty:
    print(f"\nTop {len(recommended_movies)} Empfehlungen für '{example_movie_title}':")
    print(recommended_movies)
else:
    print(f"\nKeine oder nicht genug Empfehlungen für '{example_movie_title}' gefunden (Film eventuell nicht im Datensatz oder Tippfehler).")

# Weiteres Beispiel
example_movie_title_2 = 'Die nackte Kanone 33 1/3' # Ein anderer Film
print(f"\nGeneriere Empfehlungen für: '{example_movie_title_2}'")
# HIER: df durch df_movies ersetzt
recommended_movies_2 = get_recommendations_nn(example_movie_title_2, df_movies, nn_model, X, num_recommendations=5)
if not recommended_movies_2.empty:
    print(f"\nTop {len(recommended_movies_2)} Empfehlungen für '{example_movie_title_2}':")
    print(recommended_movies_2)
else:
    print(f"\nKeine oder nicht genug Empfehlungen für '{example_movie_title_2}' gefunden (Film eventuell nicht im Datensatz oder Tippfehler).")


Generiere Empfehlungen für: 'Avatar 2: The Way Of Water'

Top 5 Empfehlungen für 'Avatar 2: The Way Of Water':
                               Titel  Ähnlichkeit                     Genres  \
0     Avatar - Aufbruch nach Pandora     0.775669          Abenteuer, Sci-Fi   
1    Planet der Affen 4: New Kingdom     0.427978  Action, Abenteuer, Sci-Fi   
2              Aliens - Die Rückkehr     0.422591             Action, Sci-Fi   
3  Terminator 2 - Tag der Abrechnung     0.418245             Action, Sci-Fi   
4       Planet der Affen 3: Survival     0.410812  Action, Abenteuer, Sci-Fi   

           Regie  Laufzeit (Min.)  Bewertung (Original)  
0  James Cameron            162.0              3.986667  
1       Wes Ball            145.0              3.566667  
2  James Cameron            137.0              4.032258  
3  James Cameron            137.0              4.666667  
4    Matt Reeves            140.0              3.290323  

Generiere Empfehlungen für: 'Die nackte Kanone 33 1/3'

To

In [27]:
# Importe für Gradio (sollten bereits oben im Skript importiert sein, hier zur Klarheit wiederholt)
import gradio as gr
import pandas as pd
import numpy as np
from fuzzywuzzy import process
from fuzzywuzzy import fuzz

print("\n--- Starte Gradio Interface Setup mit verbesserter Feedback-Anzeige ---")

# --- 3. Gradio Interface erstellen und starten ---
# Wir ändern die Wrapper-Funktion, um sowohl Text als auch DataFrame zurückzugeben
def generate_and_display_recommendations_for_gradio(movie_title: str) -> tuple[str, pd.DataFrame]:
    """
    Wrapper-Funktion für Gradio, die Empfehlungen generiert, das Ähnlichkeitsformat anpasst
    und den gefundenen Filmtitel zurückgibt.
    """
    # Zuerst versuchen wir, den Titel zu matchen
    all_titles = df_movies['titel'].tolist()
    # Entferne den 'cutoff'-Parameter hier
    best_match = process.extractOne(movie_title, all_titles, scorer=fuzz.token_set_ratio) 

    # Überprüfe den Ähnlichkeits-Score manuell, da 'cutoff' nicht direkt in extractOne ist
    if best_match is None or best_match[1] < 75: # Hier wird der Schwellenwert von 75% angewendet
        feedback_message = f"Film '{movie_title}' konnte nicht eindeutig im Datensatz gefunden werden (beste Übereinstimmung: '{best_match[0]}' mit {best_match[1]}%). Bitte versuchen Sie einen anderen Titel oder eine präzisere Schreibweise."
        return feedback_message, pd.DataFrame(columns=['Titel', 'Ähnlichkeit', 'Genres', 'Regie', 'Laufzeit (Min.)', 'Bewertung (Original)'])
    
    exact_movie_title = best_match[0]
    matched_score = best_match[1]
    
    feedback_message = f"Empfehlungen für: **{exact_movie_title}** (Ihre Eingabe: '{movie_title}', Ähnlichkeit: {matched_score}%)"

    recommendations_df = get_recommendations_nn(
        movie_title=exact_movie_title, # Hier übergeben wir den exakt gefundenen Titel an die Funktion
                                        # damit sie intern nicht noch einmal fuzzy matching machen muss
        df_movies_original=df_movies,
        nn_model=nn_model,
        feature_matrix_X=X,
        num_recommendations=5
    )

    if recommendations_df.empty:
        return feedback_message, pd.DataFrame(columns=['Titel', 'Ähnlichkeit', 'Genres', 'Regie', 'Laufzeit (Min.)', 'Bewertung (Original)'])
    else:
        # HIER: Die Ähnlichkeit auf zwei Nachkommastellen runden
        recommendations_df['Ähnlichkeit'] = recommendations_df['Ähnlichkeit'].apply(lambda x: f"{x:.2f}") 
        recommendations_df['Bewertung (Original)'] = recommendations_df['Bewertung (Original)'].apply(lambda x: f"{x:.1f}")
        return feedback_message, recommendations_df

# Gradio Interface erstellen
# inputs: Was der Benutzer eingeben soll (ein Textfeld)
# outputs: WAS DIE FUNKTION ZURÜCKGIBT (jetzt ein Text und ein DataFrame)
iface = gr.Interface(
    fn=generate_and_display_recommendations_for_gradio,
    inputs=gr.Textbox(lines=1, placeholder="Geben Sie einen Filmtitel ein (z.B. Terminator)", label="Filmtitel"),
    outputs=[
        gr.Markdown(label="Gefundener Film & Status"), # Für die Textnachricht
        gr.DataFrame(headers=["Titel", "Ähnlichkeit", "Genres", "Regie", "Laufzeit (Min.)", "Bewertung (Original)"],
                     wrap=True,
                     column_widths=["20%", "10%", "20%", "15%", "15%", "20%"],
                     label="Filmempfehlungen") # Label für die Tabelle
    ],
    title="Film-Empfehlungssystem",
    description="Gib einen (ggf. ungenauen) Filmtitel ein, um ähnliche Filme zu finden.",
    allow_flagging="never"
)

print("Gradio Interface erstellt. Starte UI...")
iface.launch(inbrowser=True, share=False)


--- Starte Gradio Interface Setup mit verbesserter Feedback-Anzeige ---


C:\Users\rusla\AppData\Roaming\Python\Python312\site-packages\gradio\interface.py:419: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Gradio Interface erstellt. Starte UI...
* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
